<a href="https://colab.research.google.com/github/ovbystrova/Interference/blob/master/Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
!wget https://github.com/ovbystrova/Interference/raw/master/data/original_texts.csv

--2020-03-26 14:43:47--  https://github.com/ovbystrova/Interference/raw/master/data/original_texts.csv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ovbystrova/Interference/master/data/original_texts.csv [following]
--2020-03-26 14:43:48--  https://raw.githubusercontent.com/ovbystrova/Interference/master/data/original_texts.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18891291 (18M) [text/plain]
Saving to: ‘original_texts.csv.1’

original_texts.csv. 100%[===================>]  18.02M  28.3MB/s    in 0.6s    

2020-03-26 14:43:49 (28.3 MB/s) - ‘original_texts.csv.1’ saved [18891291/18891291]


In [0]:
import gensim.downloader as api
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import sklearn

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm

In [0]:
seed = 42

In [0]:
results = pd.DataFrame(columns=['model', 'preprocess', 'vectorize', 'score', 'native', 'language_background'])

In [128]:
results.columns

Index(['model', 'preprocess', 'vectorize', 'score', 'native',
       'language_background'],
      dtype='object')

In [0]:
def add_row(results_df, res_list):
  return results_df.append(dict(zip(results_df.columns, res_list)), ignore_index=True)

## Dataset

In [0]:
df = pd.read_csv('original_texts.csv', index_col='id')

In [137]:
df.head()

,native,language_background,title,text,date1,date2,level,annotated,checked
id,,,,,,,,,
1,eng,HL,"expository (non-timed, paragraph, Environmenta...",Загрязнение тяжелыми металлами Дальнегорского...,2009.0,2010.0,AM,1,1
2,NaN,NaN,–,Многие жители России пишут письма президенту М...,2000.0,2013.0,NaN,1,1
3,eng,HL,"business correspondence (non-timed, paragraph,...",Директору магазина « Адидас» М. И. Васильченк...,2009.0,2010.0,AM,1,1
4,eng,FL,"definition (non-timed, sentences, Russian in t...",&lt; Что такое аннотация?&gt;. Что такое анно...,2011.0,2012.0,IH,1,1
5,eng,FL,"summary (non-timed, paragraph, Capstone)","Вывод. Спасибо, ребята, за хорошие ответы. Я ...",2012.0,2013.0,AM,1,1


Drop useless for our task columns and NaNs

In [0]:
df = df.drop(['title', 'date1', 'date2', 'level', 'annotated', 'checked'], axis=1)
df = df.dropna()

In [139]:
df['text'] = df.text.apply(lambda x: re.sub('\&[lg]t;', '', x))
df.head()

,native,language_background,text
id,,,
1,eng,HL,Загрязнение тяжелыми металлами Дальнегорского...
3,eng,HL,Директору магазина « Адидас» М. И. Васильченк...
4,eng,FL,Что такое аннотация?. Что такое аннотация ( ...
5,eng,FL,"Вывод. Спасибо, ребята, за хорошие ответы. Я ..."
6,eng,FL,Записи. Первая проблема- что производить? втор...


In [84]:
df['native'].value_counts(normalize=True)

eng    0.401660
jap    0.200639
fin    0.157216
fr     0.063218
kaz    0.063091
ger    0.036143
kor    0.025160
swe    0.022733
ita    0.014687
taj    0.003831
nor    0.003576
chi    0.003065
ser    0.002427
dut    0.002299
est    0.000255
Name: native, dtype: float64

In [140]:
le = LabelEncoder()
le.fit(df.native.values)
list(le.classes_)
df['target_native'] = le.transform(df['native'].values)
df.head()

,native,language_background,text,target_native
id,,,,
1,eng,HL,Загрязнение тяжелыми металлами Дальнегорского...,2
3,eng,HL,Директору магазина « Адидас» М. И. Васильченк...,2
4,eng,FL,Что такое аннотация?. Что такое аннотация ( ...,2
5,eng,FL,"Вывод. Спасибо, ребята, за хорошие ответы. Я ...",2
6,eng,FL,Записи. Первая проблема- что производить? втор...,2


In [0]:
# list(le.inverse_transform([2, 2, 1])))

In [87]:
df['language_background'].value_counts(normalize=True)

FL    0.689144
HL    0.310856
Name: language_background, dtype: float64

In [141]:
le_lb = LabelEncoder()
le_lb.fit(df.language_background.values)
list(le_lb.classes_)
df['target_lb'] = le_lb.transform(df['language_background'].values)
df.head()

,native,language_background,text,target_native,target_lb
id,,,,,
1,eng,HL,Загрязнение тяжелыми металлами Дальнегорского...,2,1
3,eng,HL,Директору магазина « Адидас» М. И. Васильченк...,2,1
4,eng,FL,Что такое аннотация?. Что такое аннотация ( ...,2,0
5,eng,FL,"Вывод. Спасибо, ребята, за хорошие ответы. Я ...",2,0
6,eng,FL,Записи. Первая проблема- что производить? втор...,2,0


## TF-IDF

### on text

#### native

In [0]:
vec = TfidfVectorizer(lowercase=True, tokenizer=None, analyzer='word', stop_words='english', 
                      ngram_range=(1,3), max_df=0.9, min_df=10, 
                      use_idf=True, smooth_idf=True, norm='l2')

xs = vec.fit_transform(df['text'])
ys_n = df.target_native.values
ys_lb = df.target_lb.values

In [0]:
x_train, x_test, y_train_n, y_test_n = train_test_split(xs, ys_n,
                                                    test_size=0.2,
                                                    random_state=seed,
                                                    stratify=ys_n)

In [91]:
%%time

model_n = LogisticRegressionCV(Cs=[10, 1, 0.1], cv=3, penalty='l2', 
                             multi_class='multinomial',
                             scoring='f1_macro', n_jobs=-1, verbose=True, random_state=seed)
model_n.fit(x_train, y_train_n)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   49.2s finished


CPU times: user 14.8 s, sys: 9.54 s, total: 24.4 s
Wall time: 1min 1s


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [92]:
y_pred_n = model_n.predict(x_test)
metrics.f1_score(y_test_n, y_pred_n, average='macro')

0.5813428627511351

In [93]:
metrics.accuracy_score(y_test_n, y_pred_n)

0.8659003831417624

#### language background


In [0]:
x_train, x_test, y_train_lb, y_test_lb = train_test_split(xs, ys_lb,
                                                    test_size=0.2,
                                                    random_state=seed,
                                                    stratify=ys_lb)

In [95]:
%%time

model_lb = LogisticRegressionCV(Cs=[10, 1, 0.1], cv=3, penalty='l2', 
                             multi_class='multinomial',
                             scoring='f1_macro', n_jobs=-1, verbose=True, random_state=seed)
model_lb.fit(x_train, y_train_lb)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    5.2s finished


CPU times: user 2.83 s, sys: 2 s, total: 4.83 s
Wall time: 7.61 s


In [96]:
y_pred_lb = model_lb.predict(x_test)
metrics.f1_score(y_test_lb, y_pred_lb, average='macro')

0.8679522554098691

In [97]:
metrics.accuracy_score(y_test_lb, y_pred_lb)

0.8895274584929758

#### add results

In [129]:
results = add_row(results, ['logreg', None, 'tf-idf', 'f1', metrics.f1_score(y_test_n, y_pred_n, average='macro'), metrics.f1_score(y_test_lb, y_pred_lb, average='macro')])
results = add_row(results, ['logreg', None, 'tf-idf', 'accuracy', metrics.accuracy_score(y_test_n, y_pred_n), metrics.accuracy_score(y_test_lb, y_pred_lb)])
results

,model,preprocess,vectorize,score,native,language_background
0,logreg,None,tf-idf,f1,0.581343,0.867952
1,logreg,None,tf-idf,accuracy,0.865900,0.889527


### on tokenized

#### native

In [0]:
tokenize = lambda x: re.findall('\w+', x.lower())

In [0]:
vec2 = TfidfVectorizer(lowercase=True, tokenizer=tokenize, analyzer='word', 
                      ngram_range=(1,3), max_df=0.9, min_df=10, 
                      use_idf=True, smooth_idf=True, norm='l2')
xs_token = vec2.fit_transform(df['text'])

In [0]:
x_train_token, x_test_token, y_train_n, y_test_n = train_test_split(xs_token, ys_n,
                                                    test_size=0.2,
                                                    random_state=seed,
                                                    stratify=ys_n)

In [101]:
%%time
fi = LogisticRegression(C=1, penalty='l1', solver='saga', multi_class='multinomial', n_jobs=-1, random_state=seed)


model_token_n = make_pipeline(
    SelectFromModel(fi, threshold="mean", max_features=20000),
    LogisticRegressionCV(Cs=[10, 1, 0.1], cv=3, penalty='l2', 
                             multi_class='multinomial',
                             scoring='f1_macro', n_jobs=-1, verbose=True, random_state=seed)
)
model_token_n.fit(x_train_token, y_train_n)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    6.3s finished


CPU times: user 21.1 s, sys: 22.5 ms, total: 21.1 s
Wall time: 27.5 s


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [102]:
y_pred_token_n = model_token_n.predict(x_test_token)
metrics.f1_score(y_test_n, y_pred_token_n, average='macro')

0.4773317920457982

In [103]:
metrics.accuracy_score(y_test_n, y_pred_token_n)

0.8218390804597702

#### language background

In [0]:
x_train_token, x_test_token, y_train_lb, y_test_lb = train_test_split(xs_token, ys_lb,
                                                    test_size=0.2,
                                                    random_state=seed,
                                                    stratify=ys_lb)

In [158]:
%%time
fi = LogisticRegression(C=1, penalty='l1', solver='saga', multi_class='multinomial', n_jobs=-1, random_state=seed)


model_token_lb = make_pipeline(
    SelectFromModel(fi, threshold="mean", max_features=20000),
    LogisticRegressionCV(Cs=[10, 1, 0.1], cv=3, penalty='l2', 
                             multi_class='multinomial',
                             scoring='f1_macro', n_jobs=-1, verbose=True, random_state=seed)
)
model_token_lb.fit(x_train_token, y_train_lb)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    2.0s finished


CPU times: user 4.96 s, sys: 160 ms, total: 5.12 s
Wall time: 7 s


In [106]:
y_pred_token_lb = model_token_lb.predict(x_test_token)
metrics.f1_score(y_test_lb, y_pred_token_lb, average='macro')

0.7913515555864894

In [107]:
metrics.accuracy_score(y_test_lb, y_pred_token_lb)

0.8288633461047255

#### add results

In [130]:
results = add_row(results, ['logreg search', 'regex tokens', 'tf-idf', 'f1', metrics.f1_score(y_test_n, y_pred_token_n, average='macro'), metrics.f1_score(y_test_lb, y_pred_token_lb, average='macro')])
results = add_row(results, ['logreg search', 'regex tokens', 'tf-idf', 'accuracy', metrics.accuracy_score(y_test_n, y_pred_token_n), metrics.accuracy_score(y_test_lb, y_pred_token_lb)])
results

,model,preprocess,vectorize,score,native,language_background
0,logreg,None,tf-idf,f1,0.581343,0.867952
1,logreg,None,tf-idf,accuracy,0.865900,0.889527
2,logreg search,regex tokens,tf-idf,f1,0.477332,0.791352
3,logreg search,regex tokens,tf-idf,accuracy,0.821839,0.828863


## w2v

In [134]:
wv = api.load('word2vec-google-news-300')
wv['king'][:10]

[=================================================-] 99.9% 1661.7/1662.8MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


array([ 0.12597656,  0.02978516,  0.00860596,  0.13964844, -0.02563477,
       -0.03613281,  0.11181641, -0.19824219,  0.05126953,  0.36328125],
      dtype=float32)

In [0]:
def vectorize(sent, wv):
  sent_vec = []
  for w in sent:
    try:
      vec = wv[w]
      sent_vec.append(vec)
    except KeyError:
      continue
  if sent_vec:
    return np.mean(np.array(sent_vec), axis=0)
  else:
    return np.zeros(300)

In [0]:
df['tokens'] = df['text'].apply(tokenize)
df['vec'] = df['tokens'].apply(lambda text: vectorize(text, wv))

In [0]:
xs_vec = np.vstack(df.vec.values)

#### native

In [0]:
x_train_vec, x_test_vec, y_train_n, y_test_n = train_test_split(xs_vec, ys_n,
                                                    test_size=0.2,
                                                    random_state=seed,
                                                    stratify=ys_n)

In [160]:
%%time
fi = LogisticRegression(C=1, penalty='l1', solver='saga', multi_class='multinomial', n_jobs=-1, random_state=seed)


model_vec_n = make_pipeline(
    SelectFromModel(fi, threshold="mean", max_features=300),
    LogisticRegressionCV(Cs=[10, 1, 0.1], cv=3, penalty='l2', 
                             multi_class='multinomial',
                             scoring='f1_macro', n_jobs=-1, verbose=True, random_state=seed)
)
model_vec_n.fit(x_train_vec, y_train_n)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    4.4s finished


CPU times: user 23.9 s, sys: 1 s, total: 24.9 s
Wall time: 28.1 s


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [161]:
y_pred_vec_n = model_vec_n.predict(x_test_vec)
metrics.f1_score(y_test_n, y_pred_vec_n, average='macro')

0.10388611289518601

In [162]:
metrics.accuracy_score(y_test_n, y_pred_vec_n)

0.49169859514687103

#### language background

In [0]:
x_train_vec, x_test_vec, y_train_lb, y_test_lb = train_test_split(xs_vec, ys_lb,
                                                    test_size=0.2,
                                                    random_state=seed,
                                                    stratify=ys_lb)

In [168]:
%%time
fi = LogisticRegression(C=1, penalty='l1', solver='saga', multi_class='multinomial', n_jobs=-1, random_state=seed)


model_vec_lb = make_pipeline(
    SelectFromModel(fi, threshold="mean", max_features=300),
    LogisticRegressionCV(Cs=[10, 1, 0.1], cv=3, penalty='l2', 
                             multi_class='multinomial',
                             scoring='f1_macro', n_jobs=-1, verbose=True, random_state=seed)
)
model_vec_lb.fit(x_train_vec, y_train_lb)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


CPU times: user 4.43 s, sys: 150 ms, total: 4.58 s
Wall time: 5.8 s


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.3s finished


In [169]:
y_pred_vec_lb = model_vec_lb.predict(x_test_vec)
metrics.f1_score(y_test_lb, y_pred_vec_lb, average='macro')

0.4464108172598926

In [170]:
metrics.accuracy_score(y_test_lb, y_pred_vec_lb)

0.6730523627075351

#### add results

In [171]:
results = add_row(results, ['logreg search', 'regex tokens', 'w2v bow mean', 'f1', metrics.f1_score(y_test_n, y_pred_vec_n, average='macro'), metrics.f1_score(y_test_lb, y_pred_vec_lb, average='macro')])
results = add_row(results, ['logreg search', 'regex tokens', 'w2v bow mean', 'accuracy', metrics.accuracy_score(y_test_n, y_pred_vec_n), metrics.accuracy_score(y_test_lb, y_pred_vec_lb)])
results

,model,preprocess,vectorize,score,native,language_background
0,logreg,None,tf-idf,f1,0.581343,0.867952
1,logreg,None,tf-idf,accuracy,0.865900,0.889527
2,logreg search,regex tokens,tf-idf,f1,0.477332,0.791352
3,logreg search,regex tokens,tf-idf,accuracy,0.821839,0.828863
4,logreg search,regex tokens,w2v bow mean,f1,0.103886,0.446411
5,logreg search,regex tokens,w2v bow mean,accuracy,0.491699,0.673052


In [172]:
results.style.background_gradient(axis=0, cmap='Reds')

,model,preprocess,vectorize,score,native,language_background
0,logreg,None,tf-idf,f1,0.581343,0.867952
1,logreg,None,tf-idf,accuracy,0.8659,0.889527
2,logreg search,regex tokens,tf-idf,f1,0.477332,0.791352
3,logreg search,regex tokens,tf-idf,accuracy,0.821839,0.828863
4,logreg search,regex tokens,w2v bow mean,f1,0.103886,0.446411
5,logreg search,regex tokens,w2v bow mean,accuracy,0.491699,0.673052
